In [34]:
import requests
from bs4 import BeautifulSoup
import boto3
import os
import pandas as pd
from io import BytesIO, StringIO
import json

In [35]:
!pip install itables

In [36]:
from itables import init_notebook_mode, show


### Setting up credentials - might need this in ~/.aws/credentials file
#### And further maybe create an IAM role, not just storing these access key as is. 

In [39]:
    AWS_ACCESS_KEY_ID=<YOUR_ACCESS_KEY_ID>
    AWS_SECRET_ACCESS_KEY=<YOUR_SECRET_ACCESS_KEY>
source_url = 'https://download.bls.gov/pub/time.series/pr/'
bucket_name = 'bls-data-sharing'

In [40]:
s3 = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY
        )

In [41]:


def data_load_from_s3(bucket_name, object_name, file_type='csv'):
    obj = s3.get_object(Bucket=bucket_name, Key=object_name)
    
    if file_type == 'csv':
        df = pd.read_csv(BytesIO(obj['Body'].read()))
    elif file_type == 'json':
        obj = s3.get_object(Bucket='bls-data-sharing', Key='datausa_population.json')
        json_content = obj['Body'].read()
        json_data = json.loads(json_content)
            # Further processing may be needed here to flatten the JSON structure
        df = pd.json_normalize(json_data, record_path = ['data'])
    else:
        raise ValueError("Unsupported file type: Only 'csv' and 'json' are supported.")
    
    return df

current = data_load_from_s3('bls-data-sharing', 'pr.data.0.Current.csv','csv')
population = data_load_from_s3('bls-data-sharing', 'datausa_population.json', 'json')
show(population)


#### Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [42]:
min_year = 2013
max_year = 2018
incl_pop = population[(population['ID Year']>=min_year) & (population['ID Year']<=max_year)]
incl_pop_group = incl_pop.groupby("Slug Nation")['Population'].agg(['mean', 'std']).rename(columns={'mean': 'Mean Population', 'std': 'Std Population'})

incl_pop_group.reset_index(level=0,inplace=True)
show(incl_pop_group)


#### Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [43]:
#### Cleaning columns
current.columns = [col.strip() for col in current]
current.columns

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')

In [33]:
series_year_sum = current.groupby(["series_id", 'year'])['value'].agg({'sum'}).rename(columns = {'sum':'yearly_sum'}).reset_index()

series_max = series_year_sum.groupby(['series_id'])['yearly_sum'].agg({"max"}).rename(columns = {'max':'max_yearly_sum'}).reset_index()

series_max_year = pd.merge(series_max, series_year_sum, left_on = ['series_id', 'max_yearly_sum'], right_on = ['series_id', 'yearly_sum'], how ='inner')
show(series_max_year[['series_id', 'year', 'max_yearly_sum']])

#### Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)



In [44]:
### Cleaning series_id
current['series_id'] = current['series_id'].str.strip()
current_id = current[(current['series_id']=='PRS30006032') & (current['period']=='Q01')]
current_id = current_id[['series_id', 'year', 'period', 'value']]
### Casting population year to int
pop_short = population[['ID Year', 'Population']]
current_id_with_pop = pd.merge(current_id, pop_short, left_on='year', right_on='ID Year', how='left').drop(columns = {"ID Year"})
show(current_id_with_pop.sort_values('Population'))